In [ ]:
#import required packages
%matplotlib inline
import pandas as pd
from pandas.core.frame import DataFrame
import statsmodels.api as sm
import numpy as np
import matplotlib.pyplot as plt
import pylab as pl
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


In [ ]:
#import data from excel file work sheets
xls = pd.ExcelFile("C:\Users\VindhyaRani\Desktop\VINDHYA\Classes\DataProgramming\Project\CRM\Values.xlsx")
#Parse them and convert to data frames
trig_values = xls.parse('Transactions', index_col=None, na_values=['NA'])
trig_values1 = xls.parse('Demographics', index_col=None, na_values=['NA'])
data_join= pd.merge(trig_values, trig_values1,on='Customer',how = 'left')

In [ ]:
#duplicate the dataframe for manipulation
data_join1 = data_join.copy(deep=True)
x = len(data_join1)
purchase1 = [0]*x
sum1 = 0
avg_order_quant = []

In [ ]:
#for all the records in data frame lag purchase and evaluate past average quantity
for i in range(0,x):
    if (i%12 == 0):
        sum1 = 0
        avg_order_quant.append(sum1)
    else:
        purchase1[i] = data_join1.Purchase[i-1]
        sum1 = sum1 + data_join1.Order_Quantity[i-1]
        avg_order_quant.append(sum1/(data_join1.Quarter[i-1]))

#insert newly created columns into the dataframe
data_join1.insert(0,'Lag_Purchase',purchase1)
data_join1.insert(0,'Avg_Order_Quantity',avg_order_quant)
data_join2 = data_join1.copy(deep=True)

In [ ]:
#sample the data randomly into train and test data
sample1 = trig_values1.sample(n=350)
train_data = data_join1[data_join1.Customer.isin(sample1.Customer)]
train_data = train_data.sort_values(['Customer','Quarter'], ascending=[1,1])
test_data = data_join1[~data_join1.Customer.isin(sample1.Customer)]
test_data = test_data.sort_values(['Customer','Quarter'], ascending=[1,1])

In [ ]:
# drop extra columns that are not required in both train and test
train = train_data.drop('SOW', 1)
train = train_data.drop('CLV', 1)
test = test_data.drop('SOW', 1)
test = test_data.drop('CLV', 1)

In [ ]:
#Perform Logistic Regression
x_train = train[['Lag_Purchase','Avg_Order_Quantity','Ret_Expense','Ret_Expense_SQ','Gender','Married','Income','First_Purchase','Loyalty']]
y_train = train['Purchase']
x_test = test[['Lag_Purchase','Avg_Order_Quantity','Ret_Expense','Ret_Expense_SQ','Gender','Married','Income','First_Purchase','Loyalty']]
y_test = test['Purchase']
test = test.drop('Purchase', 1)
logit = sm.Logit(y_train,x_train)
result = logit.fit()

In [ ]:
#print summary
print result.summary()

In [ ]:
#print confidence intervals
print result.conf_int()

In [ ]:
# odds ratios
print np.exp(result.params)
params = result.params
conf = result.conf_int()
conf['OR'] = params
conf.columns = ['2.5%', '97.5%', 'OR']

In [ ]:
print np.exp(conf)

In [ ]:
predicted = result.predict(x_train)
y_predict = (predicted > 0.5).astype(int)
# Check accuracy
print(accuracy_score(y_train, y_predict))

In [ ]:
# Prediction and Accuracy Score on test data
predicted = result.predict(x_test)
y_test_predict = (predicted > 0.5).astype(int)
print(accuracy_score(y_test, y_test_predict))

In [ ]:
test.insert(0,'Purchase',y_test_predict)
test.to_csv('output.csv')

In [ ]:
#Visualization1
df5 = trig_values[0:60]
n = len(df5)
sumList3 = []
sum3 = 0
for i in range(0,n):
    if i%12 == 0 and i!=0:
        sum3 = df5.Order_Quantity[i]
        sumList3.append(sum3)
    else:
        sum3 += df5.Order_Quantity[i]
        sumList3.append(sum3)
sumList3

df6 = pd.DataFrame()
df6.insert(0, 'Sum_Quantity',sumList3)
df6.insert(0, 'Customer',df5.Customer)

values1 = df6.Sum_Quantity[0:12]
values2 = df6.Sum_Quantity[12:24]
values3 = df6.Sum_Quantity[24:36]
values4 = df6.Sum_Quantity[36:48]
values5 = df6.Sum_Quantity[48:60]

fig = plt.figure(figsize=(18,6))
line1 = plt.plot(range(1,13), values1, '-')
line2 = plt.plot(range(1,13), values2, '-')
line3 = plt.plot(range(1,13), values3, '-')
line4 = plt.plot(range(1,13), values4, '-')
line5 = plt.plot(range(1,13), values5, '-')
ax = plt.axes()
ax.set_title('Quarter-wise Ordered Quantity For 5 Customers')
ax.grid()
plt.xlabel('Quarters')
plt.ylabel('Ordered Quantity')
plt.legend(['C1', 'C2', 'C3', 'C4', 'C5'], loc='best')
plt.show()

In [ ]:
#Visualization2
df = DataFrame(train_data)
df = df[['Quarter','Purchase']]

df = df.sort_values(['Quarter', 'Purchase'], ascending=[1,1])
n = len(df)
df = df.reset_index()
list1 = []
list2 = []
for i in range(0, n, 350):
    sumPurchased = 0
    sumNonPurchased = 0
    for j in range(i, i+350):
        if df.Purchase[j] == 1:
            sumPurchased += 1
        else:
            sumNonPurchased += 1
    list1.append(sumPurchased)
    list2.append(sumNonPurchased)                
fig = plt.figure(figsize=(18,6))
ax = plt.axes()
line1 = plt.plot(range(1,13), list1, '-')
line2 = plt.plot(range(1,13), list2, '-')
ax.set_title('Quarter-wise Number of Purchases and Non Purchases')
ax.set_xticks([1,2,3,4,5,6,7,8,9,10,11,12])
ax.grid()
plt.xlabel('Quarters')
plt.ylabel('Purchases and Non Purchases')
plt.legend(['Purchased', 'Non-Purchased'], loc=4)
plt.show()

In [ ]:
#Visualization3
sumList = []
x = len(data_join1)
for i in range(0,x):
    if (i%12 == 0):
        if i != 0:
            sumList.append(sum1)
        sum1 = data_join1.Order_Quantity[i]
    else:
        sum1 = sum1 + data_join1.Order_Quantity[i]
sumList.append(sum1)
df2 = pd.DataFrame()
df2.insert(0, 'Sum_Quantity',sumList)
df2.insert(0, 'Customer',range(1,501))
df2 = df2.sort_values(['Sum_Quantity'], ascending=[0])
df2 = df2.head()
values = df2.Sum_Quantity[:]
widths = 0.7
colors = ['r','g','b','y','m']
fig = plt.figure(figsize=(18,6))
ax = plt.axes()
plt.bar(range(5), values, width = widths, color=colors, align = 'center', alpha=0.6)
plt.xticks(range(5), df2.Customer[:])
ax.set_title('Total ordered quantity of top 5 Customers')
ax.grid()
plt.ylabel('Total Ordered Quantity')
plt.xlabel('Customer Ids')
plt.show()

In [ ]:
#Visualization4
income_purchase = pd.crosstab(data_join.Purchase,data_join.Income)
index = np.array(income_purchase.columns.values)
list_nonpurchase = np.array(income_purchase.ix[0].values)
list_purchase = np.array(income_purchase.ix[1].values)
widths = 0.35
fig = plt.figure(figsize=(18,6))
rects1 = plt.bar(index, list_nonpurchase, widths, color = 'b', label = 'Not Purchase',alpha=0.55)
rects2 = plt.bar(index+widths, list_purchase, widths, color = 'r', label = 'Purchase',alpha=0.55)
plt.xlabel('Income')
plt.ylabel('Count')
plt.title('Purchase and Not-Purchase by Income ')
plt.xticks(index+widths, ('$30000', '$30001-$45000', '$45001-$60000', '$60001-$75000', '$75001-$90000', '>$90001'),rotation=45)
plt.legend()
plt.show()
